In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

import yfinance as yf
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)



import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot
import cufflinks as cf
cf.go_offline()

import sys
sys.path.append('/Users/educontreras/PycharmProjects/Quantitative-Finance/AlgoTrading_E_Chang')
import alpha_vantage
from Indicators.Momentum.Multi_Spectral_RSI_Divergence import MS_RSI_DIVERGENCE

In [2]:
M = MS_RSI_DIVERGENCE("COPX", "2005-01-01")
M.backtest_strategy()

[*********************100%%**********************]  1 of 1 completed


IN SAMPLE SHARPE:  2.914901934714387
OUT SAMPLE SHARPE:  0.15597083410115592
OUT OF SAMPLE LONG SHARPE 0.22514546717947478
OUT OF SAMPLE SHORT SHARPE 0.002140611540199406


In [2]:
M = MS_RSI_DIVERGENCE("NVDA", "2005-01-01")
M.compute_divergence(5, 10,20,5)
M.compute_profits(entry_level=5, trend_confirmation_days=1)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


## When are we losing money? Long-only version

Here we aiming to understand what happens when we enter a long position, when are we losing money and why?

In [3]:
mask = M.long_positions == 1
longs = M.long_positions[mask].dropna()
longs

,Long
Date,
2005-03-14,1.0
2005-03-15,1.0
2005-03-16,1.0
2005-03-23,1.0
2005-03-24,1.0
...,...
2024-02-26,1.0
2024-04-12,1.0
2024-04-15,1.0


In [4]:
fig = go.Figure()
aux_df = pd.merge(M.prices, M.prices[M.prices.index.isin(longs.index)], left_index = True, right_index = True, how = "outer")
aux_df.columns = ["Total Price", "Price when Long"]
fig.add_trace(
    go.Scatter(
        x = aux_df.index,
        y = aux_df.iloc[:, 0].values.flatten(),
        mode = "lines",
        name = "Long Equity Curve"
        
    )
)
fig.add_trace(
    go.Scatter(
        x = aux_df.index,
        y = aux_df.iloc[:, 1].values.flatten(),
        name = "Price when Long",
        line = dict(
            color = "darkgreen", width = 3
        )
    )
)
fig.show()

In [16]:
M.long_account_curve.iplot()

In [8]:
short_days = [element for element in M.short_positions[M.short_positions.diff() != 0]]


In [9]:
M.indicator["Open"]

,Lvl,Trend,Trend Change,# Days in Trend
Date,,,,
2010-04-28,0.053291,1,NaN,0
2010-04-29,-1.580854,-1,-1.0,0
2010-04-30,0.023288,1,1.0,0
2010-05-03,-1.002313,-1,-1.0,2
2010-05-04,-3.685423,-1,0.0,0
...,...,...,...,...
2024-04-15,4.264092,-1,-1.0,2
2024-04-16,-8.016923,-1,0.0,0
2024-04-17,-11.973657,-1,0.0,0


In [33]:
longs.tail()

,Long
Date,
2024-03-29,1.0
2024-03-30,1.0
2024-03-31,1.0
2024-04-01,1.0
2024-04-02,1.0


In [38]:
df = pd.merge(M.indicator["Open"].loc[:, "Lvl"], M.indicator["Close"].loc[:, "Lvl"], left_index = True, right_index = True)
df.iplot()

In [10]:
#Cuando haces diff: 1 es la entrada en long y -1 es la entrada en short

longs_entry_exit = M.long_positions.diff()
entry = longs_entry_exit[longs_entry_exit == 1].dropna().index.tolist()
exit = longs_entry_exit[longs_entry_exit == 1].dropna().index.tolist()

In [13]:
list(zip(entry,exit))

[(Timestamp('2005-02-25 00:00:00'), Timestamp('2005-02-25 00:00:00')),
 (Timestamp('2005-03-21 00:00:00'), Timestamp('2005-03-21 00:00:00')),
 (Timestamp('2005-06-09 00:00:00'), Timestamp('2005-06-09 00:00:00')),
 (Timestamp('2005-06-22 00:00:00'), Timestamp('2005-06-22 00:00:00')),
 (Timestamp('2005-08-24 00:00:00'), Timestamp('2005-08-24 00:00:00')),
 (Timestamp('2005-09-22 00:00:00'), Timestamp('2005-09-22 00:00:00')),
 (Timestamp('2005-12-05 00:00:00'), Timestamp('2005-12-05 00:00:00')),
 (Timestamp('2005-12-16 00:00:00'), Timestamp('2005-12-16 00:00:00')),
 (Timestamp('2006-01-04 00:00:00'), Timestamp('2006-01-04 00:00:00')),
 (Timestamp('2006-01-26 00:00:00'), Timestamp('2006-01-26 00:00:00')),
 (Timestamp('2006-02-16 00:00:00'), Timestamp('2006-02-16 00:00:00')),
 (Timestamp('2006-03-28 00:00:00'), Timestamp('2006-03-28 00:00:00')),
 (Timestamp('2006-07-18 00:00:00'), Timestamp('2006-07-18 00:00:00')),
 (Timestamp('2006-09-11 00:00:00'), Timestamp('2006-09-11 00:00:00')),
 (Time

In [18]:
M.long_positions

,Long
Date,
2005-01-11,0.0
2005-01-12,0.0
2005-01-13,0.0
2005-01-14,0.0
2005-01-18,0.0
...,...
2024-04-19,0.0
2024-04-22,0.0
2024-04-23,0.0


In [20]:
M.indicator

{'Close':                   Lvl  Trend  Trend Change  # Days in Trend
 Date                                                       
 2005-01-11  -0.289695      1           NaN                0
 2005-01-12  -0.966583     -1          -1.0                0
 2005-01-13   1.232080      1           1.0                2
 2005-01-14   4.237244      1           0.0                0
 2005-01-18   8.149978      1           0.0                0
 ...               ...    ...           ...              ...
 2024-04-19 -10.100628     -1           0.0                0
 2024-04-22  -8.428867      1           1.0                2
 2024-04-23  -1.873662      1           0.0                0
 2024-04-24  -0.433478      1           0.0                0
 2024-04-25   0.198151      1           0.0                0
 
 [4855 rows x 4 columns],
 'Open':                   Lvl  Trend  Trend Change  # Days in Trend
 Date                                                       
 2005-01-11  -0.343852      1           

In [21]:
M.indicator["Open"]

,Lvl,Trend,Trend Change,# Days in Trend
Date,,,,
2005-01-11,-0.343852,1,NaN,0
2005-01-12,-1.040204,-1,-1.0,1
2005-01-13,-1.631010,-1,0.0,0
2005-01-14,2.084599,1,1.0,2
2005-01-18,3.824582,1,0.0,0
...,...,...,...,...
2024-04-19,-8.807064,-1,0.0,0
2024-04-22,-12.189904,-1,0.0,0
2024-04-23,-9.388912,1,1.0,1
